# Explaining Thoth's design and basic usage

[Thoth](https://en.wikipedia.org/wiki/Thoth) is an analytics platform that helps you explore, examine and build recommendations for software packages and (not only) AI software stacks in Python. It runs on [OpenShift](https://www.openshift.com/) and allows you to use [Jupyter](http://jupyter.org/) notebooks to interact with the system. Thoth comes with packages that have some useful pre-baked code that will help you with repetative routines, querying for data in data stores or with integration with popular scientific Python packages such as [networkx](http://networkx.github.io/), [matplotlib](https://matplotlib.org/) and others.

The aim of this notebook is to create an interactive guide on how to use Thoth, how Thoth is designed, how to use its packages and how to explore system's features and abilities. All Thoth packages are open source, installable on your system via standard Python packaging system. Also, Thoth is deployable to your own cluster.

## Thoth packages

Thoth packages are designed to create a system of loosely coupled pieces that create building blocks for the whole system. This allows to reuse parts in production deployment, simplify unit and integration tests as well as reusability in notebooks such as this one following the well known Unix idea: *"Do one thing and do it well"*. Currently, there are available the following Thoth packages:

 * [thoth-adviser](https://github.com/fridex/thoth-adviser) - recommendation algorithms and recommendation engine
 * [thoth-analyzer](https://github.com/fridex/thoth-analyzer) - code that is shared across analyzers (such as solver or package-extract, see bellow)
 * [thoth-common](https://github.com/fridex/thoth-common) - a common library that shares code for cluster deployment, such as configuration of logging facilities
 * [thoth-lab](https://github.com/fridex/thoth-lab) - various helpers intended to be used with Jupyter notebooks or experiments
 * [thoth-package-extract](https://github.com/fridex/thoth-package-extract) - extraction of packages rpm or Python packages available inside images
 * [thoth-solver](https://github.com/fridex/thoth-solver) - a library providing solvers to resolve requirements such the ones as stated in Python's requirements.txt file
 * [thoth-storages](https://github.com/fridex/thoth-storages) - a library providing storage and database adapters


All of these packages are available on [PyPI](http://pypi.python.org/) under same names, installable via pip or [pipenv](https://docs.pipenv.org/) (recommended):

```
$ pipenv install thoth-storages
```

After installation you can directly issue imports of a package module. All packages are namespaced, exposing the `thoth` namespace so to import the `thoth-package-extract` package, you write the following Python import statement (dashes become underscores in imports):

In [1]:
import thoth.package_extract

As Python provides documentation using [docstrings](https://www.python.org/dev/peps/pep-0257/), you can issue the `help()` command on any Python object (package, module, class, function, ...) to see it's documentation:

In [2]:
help(thoth.package_extract)

Help on package thoth.package_extract in thoth:

NAME
    thoth.package_extract - Extraction of installed packages for project Thoth.

PACKAGE CONTENTS
    cli
    core
    exceptions
    handlers (package)
    image

DATA
    __copyright__ = 'Copyright 2018 Fridolin Pokorny'
    __license__ = 'ASL 2.0'
    __title__ = 'thoth-package-extract'

VERSION
    1.0.0rc3

AUTHOR
    Fridolin Pokorny

FILE
    /home/fpokorny/.virtualenvs/thoth-notebooks-7XLP4Rya/lib/python3.6/site-packages/thoth/package_extract/__init__.py




After the import above you can access code provided by package in a dotted form, however it is more recommended to directly import desired classes or functions:

In [3]:
from thoth.package_extract.core import extract_image

help(extract_image)

Help on function extract_image in module thoth.package_extract.core:

extract_image(image_name:str, timeout:int=None) -> dict
    Extract dependencies from an image.



Just for completeness, a plain text docstring can be also accessed via the `__doc__` attribute of any Python object:

In [4]:
extract_image.__doc__

'Extract dependencies from an image.'

As stated earlier, Python packages are reused in various Thoth components in production, but you can also use facilities provided by packages right inside Jupyter notebooks for your exepriments. For example to look for Python packages installed inside the Fedora 27 image, you create a code snipped as shown bellow:

In [5]:
result = extract_image("fedora:27")

installed_pypi_packages = []
for pypi_package in result['mercator']:
    installed_pypi_packages.append({
        'path': pypi_package['path'],
        'package': pypi_package['result'].get('name', 'UNKNOWN'),
        'version': pypi_package['result'].get('version', 'N/A')
    })

And let Jupyter notebook print results for you:

In [6]:
installed_pypi_packages

[{'package': 'iniparse',
  'path': '/usr/lib/python3.6/site-packages/iniparse-0.4-py3.6.egg-info/PKG-INFO',
  'version': '0.4'},
 {'package': 'pip',
  'path': '/usr/lib/python3.6/site-packages/pip-9.0.1.dist-info/metadata.json',
  'version': '9.0.1'},
 {'package': 'setuptools',
  'path': '/usr/lib/python3.6/site-packages/setuptools-36.2.0.dist-info/metadata.json',
  'version': '36.2.0'},
 {'package': 'six',
  'path': '/usr/lib/python3.6/site-packages/six-1.11.0.dist-info/metadata.json',
  'version': '1.11.0'}]

The implementation of the `extract_image` function provided by `thoth-package-extract` package downloads the requested image (in the example above from Docker Hub, however you can explicitly state Docker registry to download image), extracts its layers to a temporary directory and uses the [Mercator](https://github.com/fabric8-analytics/mercator-go) tool to look for artifacts that describe installed packages (such as `setup.py` or `metadata.json` in case of Python).

In case you would like to see `extract_image` on the implementation level, you can dive directly into sources as all packages are hosted on GitHub. However you can also access interactive package documentation that is automatically generated from sources of all Thoth packages. The documentation is hosted at http://fridex.github.io/thoth/. This documentation is devided into packages and modules as Thoth provides them. One of the main benefits of the online documentation is the fact, that it is searchable and can directly link you to source code, if more details are needed.